In [22]:
import torch

# Force CPU for safety
device = torch.device("cpu")

# A1 Example

# Create tensors
a = torch.tensor([1.0, 2.0, 3.0]).to(device)
b = torch.tensor([[0, 1], [2, 3]]).to(device)

# Basic operations
c = a + 2               # elementwise add
d = b * b               # elementwise multiply
e = torch.matmul(b, b)  # matrix multiply

f = torch.arange(4).reshape([2,2]).to(device)

print("a:", a)
print("b:\n", b)
print("c:", c)
print("d:\n", d)
print("e:\n", e)
print("f:\n", f) 
print('b == f:\n', b == f)

a: tensor([1., 2., 3.])
b:
 tensor([[0, 1],
        [2, 3]])
c: tensor([3., 4., 5.])
d:
 tensor([[0, 1],
        [4, 9]])
e:
 tensor([[ 2,  3],
        [ 6, 11]])
f:
 tensor([[0, 1],
        [2, 3]])
b == f:
 tensor([[True, True],
        [True, True]])


In [29]:
# A1 Problem

# 1) Create a random 3×3 matrix
x = torch.rand(9).reshape(3, 3)   # same as torch.rand(3,3)

# 2) Invert it
x_inv = torch.linalg.inv(x)   # equivalent to torch.inverse(x)

# 3) Verify the result is (approximately) the identity
I_approx = x @ x_inv

print("x:\n", x)
print("\nx_inv:\n", x_inv)
print("\nx @ x_inv:\n", I_approx)

x:
 tensor([[0.7244, 0.2596, 0.2591],
        [0.1303, 0.6867, 0.4001],
        [0.8548, 0.4090, 0.0654]])

x_inv:
 tensor([[ 0.8619, -0.6463,  0.5379],
        [-2.4216,  1.2645,  1.8595],
        [ 3.8758,  0.5397, -3.3668]])

x @ x_inv:
 tensor([[ 1.0000e+00,  2.6772e-08,  2.7783e-08],
        [ 1.4471e-08,  1.0000e+00,  6.8511e-08],
        [-5.9605e-08,  5.2154e-08,  1.0000e+00]])


In [51]:
# A2 Example

x = torch.tensor(1.5, requires_grad = True)

y = x**3 + 2*(x**2) + 3*x + 10

y.backward()

print(y.item(), x.grad.item())

22.375 15.75


In [65]:
# A2 Problem

v = torch.tensor([2.0, 3.0], requires_grad=True)
x, y = v
function = x**2 * y + torch.sin(y)

function.backward() 

print(v.grad)

tensor([12.0000,  3.0100])


In [84]:
import torch.nn as nn
import torch.nn.functional as F

# A3 Example

# 1) Define the model
class SimpleNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

# 2) Instantiate and move to device
device = torch.device("cpu")
model = SimpleNet(input_dim=10, hidden_dim=32, output_dim=4).to(device)

# 3) Create dummy data
batch_size = 5
dummy_inputs = torch.randn(batch_size, 10).to(device)
dummy_targets = torch.randint(0, 4, (batch_size,)).to(device)

# 4) Forward pass
logits = model(dummy_inputs)    # shape (5, 4)
print("Logits:", logits)

# 5) Compute loss
criterion = nn.CrossEntropyLoss()
loss = criterion(logits, dummy_targets)
print("Loss:", loss.item())

# 6) Backward pass
model.zero_grad()
loss.backward()

# 7) Inspect a gradient
print("Gradient of fc1 weights:", model.fc1.weight.grad.norm())
print(dummy_targets)

Logits: tensor([[ 0.3384,  0.0377, -0.4855, -0.0811],
        [ 0.3641, -0.7139,  0.0149, -0.5664],
        [ 0.4810,  0.2774,  0.0405,  0.2808],
        [ 0.5453, -0.0099,  0.1290, -0.1713],
        [ 0.0178, -0.0374, -0.2701, -0.1868]], grad_fn=<AddmmBackward0>)
Loss: 1.328674554824829
Gradient of fc1 weights: tensor(0.4944)
tensor([3, 2, 2, 0, 1])


In [75]:
# dummy_inputs: shape (5, 10)
print(dummy_inputs.shape)  # → torch.Size([5, 10])

# After passing through fc1 (10 → 32)
h = model.fc1(dummy_inputs)
print(h.shape)             # → torch.Size([5, 32])

# After ReLU + Dropout (still batch‑first)
h2 = model.dropout(F.relu(h))
print(h2.shape)            # → torch.Size([5, 32])

# Final logits (32 → 4)
logits = model.fc2(h2)
print(logits.shape)        # → torch.Size([5, 4])

torch.Size([5, 10])
torch.Size([5, 32])
torch.Size([5, 32])
torch.Size([5, 4])


In [83]:
x = torch.randn(5, 5).to(device)
y = torch.rand(5, 32).to(device)
x @ y

tensor([[-3.9840e-01, -7.0132e-01,  7.1361e-01,  6.0968e-01,  5.0306e-01,
          3.2268e-01, -1.9605e-01, -7.8206e-01,  3.7857e-01,  1.1249e+00,
          6.4447e-01,  1.3358e-01,  1.4694e-01, -7.9410e-01,  1.2440e-01,
          2.9275e-01,  2.6654e-01, -2.9871e-01,  1.7400e+00, -5.7731e-01,
         -2.4781e-02,  1.1566e+00,  1.9961e-01, -5.9584e-01,  1.2809e-02,
         -4.8069e-01, -3.6243e-01,  1.2256e+00,  8.9545e-03, -1.9665e-01,
         -9.0764e-01,  5.7992e-01],
        [ 1.4939e+00,  1.1682e+00,  2.3110e+00,  6.5694e-01,  8.8104e-01,
          3.9764e+00,  3.3569e+00,  2.4463e+00,  1.3139e+00,  3.2352e+00,
          2.0776e+00,  2.0589e+00,  3.2217e+00,  2.6365e+00,  3.3993e+00,
          2.0104e+00,  2.0942e+00,  1.3410e+00,  2.5867e+00,  2.6264e+00,
          3.2745e+00,  1.9748e+00,  2.4068e+00,  1.4588e+00,  1.6310e+00,
          1.9032e+00,  2.3281e+00,  1.6131e+00,  3.0122e+00,  1.6729e+00,
          1.6592e+00,  7.8077e-01],
        [-8.7182e-01,  4.0196e-01, -1.09